In [391]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
from collections import Counter
import warnings

In [405]:
# loading the imdb_full_info_df and inflation_date
imdb_full_info_df = pd.read_csv('imdb_full_info_df.csv')
inflation_df = pd.read_csv('inflation_data.csv')

In [406]:
inflation_df = inflation_df.sort_values('year', ascending=False)
inflation_df.drop(20, axis=0, inplace=True)
inflation_df

,year,dollar,inflation
19,2019,1.48,0.0176
18,2018,1.46,0.0249
17,2017,1.42,0.0213
16,2016,1.39,0.0126
15,2015,1.38,0.0012
14,2014,1.37,0.0162
13,2013,1.35,0.0146
12,2012,1.33,0.0207
11,2011,1.31,0.0316
10,2010,1.27,0.0164


In [390]:
for i in range(0, len(inflation_df)):
    

{2000: 1.0,
 2001: 1.0285,
 2002: 1.0447503,
 2003: 1.06857060684,
 2004: 1.096994584981944,
 2005: 1.134182701412832,
 2006: 1.1708168026684664,
 2007: 1.2041850815445176,
 2008: 1.2504257886758272,
 2009: 1.2459242558365942,
 2010: 1.2663574136323144,
 2011: 1.3063743079030956,
 2012: 1.3334162560766896,
 2013: 1.3528841334154091,
 2014: 1.3748008563767387,
 2015: 1.376450617404391,
 2016: 1.3937938951836861,
 2017: 1.4234817051510988,
 2018: 1.458926399609361,
 2019: 1.4846035042424859,
 2020: 1.5028641273446683}

In [373]:
imdb_full_info_df['release_date'][0].split('-')[0]

'2019'

In [388]:
# adjusting inflation for budget and worldwide_gross
budget_adj = []
worldwide_gross_adj = []

for i in range(0,len(imdb_full_info_df)):
    release_year = int(imdb_full_info_df['release_date'][i].split('-')[0])
    inflation_comp = inflation[release_year]
    budget_adj.append(round(imdb_full_info_df['budget'][i]*inflation_comp))
    worldwide_gross_adj.append(round(imdb_full_info_df['worldwide_gross'][i]*inflation_comp))

imdb_full_info_df['budget_adj'] = budget_adj
imdb_full_info_df['worldwide_gross_adj'] = worldwide_gross_adj

In [389]:
imdb_full_info_df

,release_date,budget,domestic_gross,worldwide_gross,title,imdb_rating,cast_1,cast_2,cast_3,movie_url,imdb_id,director,rating_check,runtime,genre,director.1,budget_adj,worldwide_gross_adj
0,2019-04-23,400000000,858373000,2797800564,Avengers: Endgame,8.4,Robert Downey Jr.,Chris Evans,Mark Ruffalo,https://www.imdb.com/title/tt4154796/,tt4154796,Anthony Russo,8.4,['181'],"['Action', 'Adventure', 'Drama', 'Sci-Fi']",Anthony Russo,593841402,4153624521
1,2011-05-20,379000000,241071802,1045713802,Pirates of the Caribbean: On Stranger Tides,6.6,Johnny Depp,Penélope Cruz,Ian McShane,https://www.imdb.com/title/tt1298650/,tt1298650,Rob Marshall,6.6,['137'],"['Action', 'Adventure', 'Fantasy']",Rob Marshall,495115863,1366093644
2,2015-04-22,365000000,459005868,1395316979,Avengers: Age of Ultron,7.3,Robert Downey Jr.,Chris Evans,Mark Ruffalo,https://www.imdb.com/title/tt2395427/,tt2395427,Joss Whedon,7.3,['141'],"['Action', 'Adventure', 'Sci-Fi']",Joss Whedon,502404475,1920584917
3,2015-12-16,306000000,936662225,2064615817,Star Wars Episode VII: The Force Awakens,7.9,Daisy Ridley,John Boyega,Oscar Isaac,https://www.imdb.com/title/tt2488496/,tt2488496,J.J. Abrams,7.9,['138'],"['Action', 'Adventure', 'Sci-Fi']",J.J. Abrams,421193889,2841841716
4,2018-04-25,300000000,678815482,2048359754,Avengers: Infinity War,8.5,Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,https://www.imdb.com/title/tt4154756/,tt4154756,Anthony Russo,8.5,['149'],"['Action', 'Adventure', 'Sci-Fi']",Anthony Russo,437677920,2988406121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,2011-09-30,5000000,223838,5476793,Tucker & Dale vs. Evil,7.5,Tyler Labine,Alan Tudyk,Katrina Bowden,https://www.imdb.com/title/tt1465522/,tt1465522,Eli Craig,7.5,['89'],"['Comedy', 'Horror']",Eli Craig,6531872,7154742
3310,2007-11-16,5000000,65388,861325,Redacted,6.1,Patrick Carroll,Rob Devaney,Izzy Diaz,https://www.imdb.com/title/tt0937237/,tt0937237,Brian De Palma,6.1,['90'],"['Crime', 'Thriller', 'War']",Brian De Palma,6020925,1037195
3311,2014-10-17,5000000,56001,584499,Rudderless,7.4,Billy Crudup,Anton Yelchin,Felicity Huffman,https://www.imdb.com/title/tt1798243/,tt1798243,William H. Macy,7.4,['105'],"['Comedy', 'Drama', 'Music']",William H. Macy,6874004,803570
3312,2012-05-17,5000000,0,9109597,Mientras duermes,7.2,Luis Tosar,Marta Etura,Alberto San Juan,https://www.imdb.com/title/tt1437358/,tt1437358,Jaume Balagueró,7.2,['102'],"['Crime', 'Drama', 'Thriller']",Jaume Balagueró,6667081,12146885


In [344]:
# extracting runtime values
runtime = []
for i in imdb_full_info_df['runtime']:
    runtime.append(int(i[2:][:-2]))
    
imdb_full_info_df['runtime'] = runtime

In [349]:
# dropping irrelevant columns
imdb_full_info_df.drop(columns=['release_date', 'domestic_gross', 'title', 'cast_1', 'cast_2', 'cast_3', 'movie_url', 'imdb_id', 'director', 'director.1','rating_check'], inplace=True)

In [339]:
# one hot encoding for genre (multiple values)
x = imdb_full_info_df['genre'].apply(eval)
x = x.apply(Counter)
x = pd.json_normalize(x)
x = x.replace(np.nan, 0)
x = x.astype('int')

In [340]:
# merging two dataframes
model1_df = pd.concat([imdb_full_info_df, x], axis=1)
model1_df

,budget,worldwide_gross,imdb_rating,runtime,genre,Action,Adventure,Drama,Sci-Fi,Fantasy,...,Crime,Western,History,Sport,Horror,Music,War,Biography,Documentary,Short
0,400000000,2797800564,8.4,181,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,379000000,1045713802,6.6,137,"['Action', 'Adventure', 'Fantasy']",1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,365000000,1395316979,7.3,141,"['Action', 'Adventure', 'Sci-Fi']",1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,306000000,2064615817,7.9,138,"['Action', 'Adventure', 'Sci-Fi']",1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,300000000,2048359754,8.5,149,"['Action', 'Adventure', 'Sci-Fi']",1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,5000000,5476793,7.5,89,"['Comedy', 'Horror']",0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3310,5000000,861325,6.1,90,"['Crime', 'Thriller', 'War']",0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3311,5000000,584499,7.4,105,"['Comedy', 'Drama', 'Music']",0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3312,5000000,9109597,7.2,102,"['Crime', 'Drama', 'Thriller']",0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [315]:
# one hot encoding for genre (multiple values) - alternative
genre = []
for i in imdb_full_info_df['genre']:
    genre.append((i[2:][:-2]).replace("'","").replace(', ',','))
    
imdb_full_info_df['genre'] = genre

# finding unique values for genre
genre_cat = imdb_full_info_df['genre'].value_counts().reset_index()
genre_cat.head

genre_cat.rename(columns={'index':'genre_name', 'genre':'count'}, inplace=True)

genre_cat['genre_name'] = genre_cat['genre_name'].str.split(',')

df1 = []
for i in range(genre_cat.shape[0]):
    for j in genre_cat['genre_name'][i]:
        df1.append(j)
        
# creating a list of unique genres
list1 = list(set(df1))
list1

# creating a new dataframe with genre columns
df2 = pd.DataFrame(columns=list1)

# merging two dataframes
df3 = pd.concat([imdb_full_info_df, df2], axis=1)
df3

# filling binary values for genre columns
for i in range(0, len(df3)):
    for j in range(0, len(list1)):
        if df3.columns[5+j]in df3['genre'][i]:
            df3[df3.columns[5+j]][i] = 1
        else:
            df3[df3.columns[5+j]][i] = 0

In [277]:
# X, y split
X = df3.drop('worldwide_gross', axis=1)
y = df3['worldwide_gross']

In [281]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 100)

In [282]:
# standardizing the data
std_scaler = StandardScaler().fit(X_train)
X_train_scaled = std_scaler.transform(X_train)

In [283]:
X_test_scaled = std_scaler.transform(X_test)

In [284]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [285]:
model.coef_

array([ 1.48606886e+08,  4.68340476e+07,  9.04901343e+06,  4.26395635e+06,
        3.85628204e+03,  1.05685015e+07, -1.91296318e+06, -5.51420546e+06,
        2.13069253e+06,  7.47738759e+05, -3.53057517e+06,  1.15543201e+07,
       -6.36616581e+06, -8.51088006e+06,  2.94787470e+05, -2.15155072e+07,
        1.29329283e+05, -6.90774294e+06,  4.30706245e+06, -1.01092988e+07,
       -7.53272203e+06, -2.55763203e+06, -5.98759908e+06, -9.06090542e+06,
        5.95589466e+05])

In [287]:
model.score(X_train_scaled, y_train)

0.6274430751260955

In [286]:
model.score(X_test_scaled, y_test)

0.6158188013122146

In [288]:
features_importance = pd.DataFrame(data = {
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_)
})
features_importance = features_importance.sort_values(by='Importance', ascending=False)

In [289]:
features_importance

,Attribute,Importance
0,budget,1.486069e+08
1,imdb_rating,4.683405e+07
15,Drama,2.151551e+07
11,Horror,1.155432e+07
5,Adventure,1.056850e+07
19,Western,1.010930e+07
23,Action,9.060905e+06
2,runtime,9.049013e+06
13,Thriller,8.510880e+06
20,History,7.532722e+06


In [299]:
# checking the model prediction
y_pred = model.predict(X_test_scaled)

In [302]:
R2 = r2_score(y_test, y_pred)
R2

0.6158188013122146

In [305]:
Adj_R2 = 1-((1-R2)*(len(y_test)-1)/((len(y_test)-X_test.shape[1]-1)))
Adj_R2

0.6007410462616736

In [304]:
X_test_scaled

array([[ 0.13381509,  0.04459814, -0.93693982, ..., -0.3248147 ,
         1.53759982, -0.53620725],
       [-0.51235066,  1.26977595,  0.74326293, ...,  3.07867843,
        -0.65036428, -0.53620725],
       [-0.637415  ,  0.65718705, -0.12393849, ..., -0.3248147 ,
        -0.65036428,  1.86495055],
       ...,
       [-0.84585556,  1.37187411, -0.17813858, ..., -0.3248147 ,
        -0.65036428, -0.53620725],
       [-0.78332339,  1.06557965,  0.14706195, ..., -0.3248147 ,
        -0.65036428, -0.53620725],
       [-0.66868108, -0.56799077, -0.23233867, ..., -0.3248147 ,
        -0.65036428, -0.53620725]])